# Zylentrix Student Engagement Analysis

## Background
Zylentrix runs an online training platform where students enrol in different learning tracks (e.g., Digital Marketing, Python, UI/UX, etc.). The platform
records student behaviour, such as logins, course completions, time spent on lessons, and feedback.

## Data
The system captures data one student enrollment in different courses, their activity in different courses and feedback they give regarding different courses. Based on these system produce `.csv` files as the following:

1. `students.csv` – Contains student info (ID, Name, Age, Gender, Location, Enrolment Date)
2. `course_activity.csv` – Tracks user activity (Student ID, Course ID, Date, Time Spent [mins], Completion %)
3. `feedback.csv` – Records feedback (Student ID, Course ID, Rating [1-5], Feedback Text)

## Prepare Data
To begin with the analysis, first load the necessary libraries and then load the `.csv` files in data frames. For this analysis we will be using `pandas` library for data preparation, cleaning and transformation where needed.

In [36]:
# load libraries
import pandas as pd

# load csv files in data frames
students_df = pd.read_csv('data/students.csv')
course_activity_df = pd.read_csv('data/course_activity.csv')
feedback_df = pd.read_csv('data/feedback.csv')

Now as the data has been loaded, lets have a peek.

In [37]:

# quick look at the data frames
print(students_df.head())
print(course_activity_df.head())
print(feedback_df.head())

  Student_ID       Name  Age  Gender   Location Enrolment_Date
0       S001  Student_1   24  Female    Kolkata     24/11/2023
1       S002  Student_2   32   Other    Chennai     27/02/2023
2       S003  Student_3   28   Other     Mumbai     13/01/2023
3       S004  Student_4   25  Female  Bangalore     21/05/2023
4       S005  Student_5   24   Other      Delhi     06/05/2023
  Student_ID Course_ID        Date  Time_Spent_Minutes  Completion_Percentage
0       S001     PY202  05/01/2024                  90                  46.10
1       S001     DM101  28/01/2024                 155                  88.87
2       S001     UX303  28/01/2024                 177                  54.93
3       S002     PY202  03/02/2024                  45                  32.20
4       S002     UX303  15/03/2024                 119                  90.80
  Student_ID Course_ID  Rating       Feedback_Text
0       S057     UX303       2      Too fast-paced
1       S063     PY202       2  Loved the examples
2

So, from the above it is seen that the `student_df` contains details of the student's enrolled in different courses in the platform, `course_activity_df` is these students involvement in different courses, and `feedback_df` contains student feedback towards different courses. 

## Cleaning

But, this is merely an overview of the entire data set. For analysis, more details of these data sets like how large is the data set and each variables and their types are needed. Also, to get proper isight of the data and drawout meaningful observations, the data needs to be consistent and error free as much as possible and for these purposes cleaning of the data is necessay.

Let's start with the student data. First have a view of the data frame's structure.

In [38]:
# check info of the data frame
print(students_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Student_ID      100 non-null    object
 1   Name            100 non-null    object
 2   Age             100 non-null    int64 
 3   Gender          100 non-null    object
 4   Location        100 non-null    object
 5   Enrolment_Date  100 non-null    object
dtypes: int64(1), object(5)
memory usage: 4.8+ KB
None


So this data frame contains six (6) variables with hundred (100) observations. Also a matter of relief that all the observations for each of the variables are `null` free. Although, the `Enrollment_Date` field is stored as object. It is better if this is converted to `datetime` type. Also, `Student_ID` should not be duplicate. And, whitespaces needs to be removed from `Name` and `Location` data.

In [39]:
# convert Enrollment_Date to datetime
students_df['Enrolment_Date'] = pd.to_datetime(students_df['Enrolment_Date'], errors='coerce', format='%d/%m/%Y', dayfirst=True)
# remove whitespaces from Name and Location
students_df['Name'] = students_df['Name'].str.strip()
students_df['Location'] = students_df['Location'].str.strip()
# remove duplicates if any
students_df.drop_duplicates(subset='Student_ID', keep='first', inplace=True)
print(students_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Student_ID      100 non-null    object        
 1   Name            100 non-null    object        
 2   Age             100 non-null    int64         
 3   Gender          100 non-null    object        
 4   Location        100 non-null    object        
 5   Enrolment_Date  100 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 4.8+ KB
None


And the Student data is ready to be coocked. Similarly, for the course activity data, let's have a look at the data structure.

In [40]:
# check info of the data frame
print(course_activity_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 659 entries, 0 to 658
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Student_ID             659 non-null    object 
 1   Course_ID              659 non-null    object 
 2   Date                   659 non-null    object 
 3   Time_Spent_Minutes     659 non-null    int64  
 4   Completion_Percentage  659 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 25.9+ KB
None


Here also `Date` field to be converted in `datatime` type, duplicates to be removed and additionally, invalid or unrealistic data like `Time_Spent_Minutes` less than zero (0) or `Competion_Percentage` less than zero (0)/ greater than hundred (100) to be removed.

In [41]:
# convert Date to datetime
course_activity_df['Date'] = pd.to_datetime(course_activity_df['Date'], errors='coerce', format='%d/%m/%Y', dayfirst=True)
# remove duplicates if any
course_activity_df.drop_duplicates(keep='first', inplace=True)
# remove invalid data
course_activity_df = course_activity_df[course_activity_df['Time_Spent_Minutes'] >= 0]
course_activity_df = course_activity_df[(course_activity_df['Completion_Percentage'] >= 0) & 
                                        (course_activity_df['Completion_Percentage'] <= 100)]
print(course_activity_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 659 entries, 0 to 658
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Student_ID             659 non-null    object        
 1   Course_ID              659 non-null    object        
 2   Date                   659 non-null    datetime64[ns]
 3   Time_Spent_Minutes     659 non-null    int64         
 4   Completion_Percentage  659 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 25.9+ KB
None


And lastly for the feedback data. Here is the structure of the data.

In [ ]:
print(feedback_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Student_ID     80 non-null     object
 1   Course_ID      80 non-null     object
 2   Rating         80 non-null     int64 
 3   Feedback_Text  80 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.6+ KB
None


Remove whitespaces from the feedback text and invalid rating like bellow 1 or more than 5 and this will be ready. As there is no date of feedback given, we will remove the duplicates based on `Student_ID` and `Course_ID` but this time we will keep the last occurence considering that to be the latest feedback by a student regarding a course.

In [44]:
# remove whitespace from Feedback_Text
feedback_df['Feedback_Text'] = feedback_df['Feedback_Text'].str.strip()
# remove invalid ratings
feedback_df = feedback_df[feedback_df['Rating'].between(1, 5)]
# remove duplicates if any
feedback_df.drop_duplicates(subset=['Student_ID', 'Course_ID'], keep='last', inplace=True)
print(feedback_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Student_ID     80 non-null     object
 1   Course_ID      80 non-null     object
 2   Rating         80 non-null     int64 
 3   Feedback_Text  80 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.6+ KB
None


It is a good thing to see that removing duplicates or invalid data from the data frames does not shrink the size, i. e., the observations are intact and this qualifies as good data.

Now the cleaning is done, analysis can be started.